In [ ]:
#Day1

from django.db import models
from django.conf import settings
from django.utils import timezone
from django.core.validators import MinValueValidator, MaxValueValidator, RegexValidator
from django.db.models.signals import post_save
from django.dispatch import receiver
import uuid
import string
import random

# Subscription plan types
class SubscriptionPlan(models.Model):
    PLAN_TYPES = (
        ('FREE', 'Personal Free'),
        ('PRO', 'Personal Pro'),
        ('MAX', 'Personal Max'),
        ('TEAM', 'Business Team'),
        ('ENTERPRISE', 'Business Enterprise'),
    )
    
    name = models.CharField(max_length=50, choices=PLAN_TYPES, unique=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    ai_chat_limit = models.PositiveIntegerField(default=0)
    description = models.TextField()
    is_active = models.BooleanField(default=True)
    
    # Features flags for different plans
    access_tutorials = models.BooleanField(default=False)
    access_library = models.BooleanField(default=False)
    access_news = models.BooleanField(default=False)
    access_premium = models.BooleanField(default=False)
    
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    def __str__(self):
        return self.get_name_display()
    
    class Meta:
        indexes = [
            models.Index(fields=['name']),
        ]

def generate_referral_code():
    """Generate a unique 16-character referral code"""
    chars = string.ascii_uppercase + string.digits
    while True:
        code = ''.join(random.choice(chars) for _ in range(16))
        if not UserProfile.objects.filter(referral_code=code).exists():
            return code

class UserProfile(models.Model):
    user = models.OneToOneField(settings.AUTH_USER_MODEL, on_delete=models.CASCADE, related_name='profile')
    credits = models.PositiveIntegerField(default=0)
    subscription = models.ForeignKey(
        SubscriptionPlan, 
        on_delete=models.SET_NULL,
        null=True, 
        blank=True,
        related_name='subscribers'
    )
    
    # Referral system
    referral_code = models.CharField(
        max_length=16, 
        unique=True, 
        default=generate_referral_code,
        validators=[
            RegexValidator(
                regex=r'^[A-Z0-9]{16}$',
                message='Referral code must be 16 characters of uppercase letters and numbers'
            )
        ]
    )
    referred_by = models.ForeignKey(
        'self', 
        on_delete=models.SET_NULL, 
        null=True, 
        blank=True, 
        related_name='referrals'
    )
    
    # Streak tracking
    current_streak = models.PositiveIntegerField(default=0)
    highest_streak = models.PositiveIntegerField(default=0)
    last_active_date = models.DateField(null=True, blank=True)
    
    # Subscription tracking
    subscription_start_date = models.DateTimeField(null=True, blank=True)
    subscription_end_date = models.DateTimeField(null=True, blank=True)
    
    # AI usage tracking
    ai_chats_used_today = models.PositiveIntegerField(default=0)
    ai_chats_last_reset = models.DateField(null=True, blank=True)
    
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    def __str__(self):
        return f"{self.user.email}'s Profile"
    
    def has_active_subscription(self):
        if not self.subscription:
            return False
        
        if self.subscription.name == 'FREE':
            return True
            
        if not self.subscription_end_date:
            return False
            
        return self.subscription_end_date > timezone.now()
    
    def can_use_ai_chat(self):
        # Check if AI chat limit reset is needed
        today = timezone.now().date()
        if self.ai_chats_last_reset != today:
            self.ai_chats_used_today = 0
            self.ai_chats_last_reset = today
            self.save()
        
        # Check if user has an active subscription and hasn't exceeded limit
        if self.has_active_subscription() and self.ai_chats_used_today < self.subscription.ai_chat_limit:
            return True
        
        # Free users can use credits instead
        return self.credits > 0
    
    def update_streak(self):
        today = timezone.now().date()
        
        # Initialize last_active_date if None
        if not self.last_active_date:
            self.last_active_date = today
            self.current_streak = 1
            self.highest_streak = 1
            self.save()
            return
        
        # Calculate days since last activity
        days_since_last_active = (today - self.last_active_date).days
        
        if days_since_last_active == 0:
            # Already updated today
            return
        elif days_since_last_active == 1:
            # Consecutive day, increase streak
            self.current_streak += 1
            if self.current_streak > self.highest_streak:
                self.highest_streak = self.current_streak
        else:
            # Streak broken
            self.current_streak = 1
        
        self.last_active_date = today
        self.save()
    
    class Meta:
        indexes = [
            models.Index(fields=['user']),
            models.Index(fields=['referral_code']),
            models.Index(fields=['subscription']),
        ]

class CreditTransaction(models.Model):
    TRANSACTION_TYPES = (
        ('INITIAL', 'Initial Free Credits'),
        ('REFERRAL', 'Referral Bonus'),
        ('STREAK', 'Streak Bonus'),
        ('USAGE', 'Content Usage'),
        ('PURCHASE', 'Credit Purchase'),
        ('AI_CHAT', 'AI Chat Usage'),
        ('REFUND', 'Refund'),
        ('ADMIN', 'Admin Adjustment'),
    )
    
    user = models.ForeignKey(settings.AUTH_USER_MODEL, on_delete=models.CASCADE, related_name='credit_transactions')
    amount = models.IntegerField()  # Positive for credits earned, negative for credits spent
    transaction_type = models.CharField(max_length=15, choices=TRANSACTION_TYPES)
    description = models.TextField()
    balance_after = models.PositiveIntegerField()
    created_at = models.DateTimeField(auto_now_add=True)
    
    class Meta:
        ordering = ['-created_at']
        indexes = [
            models.Index(fields=['user']),
            models.Index(fields=['transaction_type']),
            models.Index(fields=['created_at']),
        ]

class Subscription(models.Model):
    user = models.ForeignKey(
        settings.AUTH_USER_MODEL, 
        on_delete=models.CASCADE, 
        related_name='subscriptions'
    )
    plan = models.ForeignKey(
        SubscriptionPlan, 
        on_delete=models.PROTECT,
        related_name='plan_subscriptions'
    )
    start_date = models.DateTimeField(default=timezone.now)
    end_date = models.DateTimeField(null=True, blank=True)
    is_active = models.BooleanField(default=True)
    auto_renew = models.BooleanField(default=True)
    
    # Payment details
    razorpay_payment_id = models.CharField(max_length=100, null=True, blank=True)
    razorpay_order_id = models.CharField(max_length=100, null=True, blank=True)
    razorpay_signature = models.CharField(max_length=255, null=True, blank=True)
    
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    def __str__(self):
        return f"{self.user.email} - {self.plan.get_name_display()}"
    
    class Meta:
        indexes = [
            models.Index(fields=['user']),
            models.Index(fields=['plan']),
            models.Index(fields=['is_active']),
            models.Index(fields=['end_date']),
        ]

# Signal to create user profile on user creation
@receiver(post_save, sender=settings.AUTH_USER_MODEL)
def create_user_profile(sender, instance, created, **kwargs):
    if created:
        # Get the free plan
        free_plan = SubscriptionPlan.objects.filter(name='FREE').first()
        
        # Create the profile
        profile = UserProfile.objects.create(
            user=instance,
            subscription=free_plan,
            subscription_start_date=timezone.now(),
        )
        
        # Add initial free credits (100 as default)
        initial_credits = 100
        profile.credits = initial_credits
        profile.save()
        
        # Record the transaction
        CreditTransaction.objects.create(
            user=instance,
            amount=initial_credits,
            transaction_type='INITIAL',
            description=f'Welcome bonus: {initial_credits} credits',
            balance_after=initial_credits
        )

In [ ]:
from rest_framework import serializers
from django.contrib.auth import get_user_model
from .models import UserProfile, CreditTransaction, SubscriptionPlan, Subscription

User = get_user_model()

class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = ['id', 'email', 'first_name', 'last_name', 'date_joined']
        read_only_fields = ['id', 'email', 'date_joined']

class SubscriptionPlanSerializer(serializers.ModelSerializer):
    class Meta:
        model = SubscriptionPlan
        fields = [
            'id', 'name', 'price', 'ai_chat_limit', 'description', 
            'access_tutorials', 'access_library', 'access_news', 'access_premium'
        ]

class SubscriptionSerializer(serializers.ModelSerializer):
    plan = SubscriptionPlanSerializer(read_only=True)
    
    class Meta:
        model = Subscription
        fields = [
            'id', 'plan', 'start_date', 'end_date', 
            'is_active', 'auto_renew', 'created_at'
        ]
        read_only_fields = fields

class CreditTransactionSerializer(serializers.ModelSerializer):
    class Meta:
        model = CreditTransaction
        fields = ['id', 'amount', 'transaction_type', 'description', 'balance_after', 'created_at']
        read_only_fields = fields

class UserProfileSerializer(serializers.ModelSerializer):
    subscription_plan = SubscriptionPlanSerializer(source='subscription', read_only=True)
    has_active_subscription = serializers.BooleanField(read_only=True)
    
    class Meta:
        model = UserProfile
        fields = [
            'id', 'credits', 'subscription_plan', 'referral_code',
            'current_streak', 'highest_streak', 'last_active_date',
            'subscription_start_date', 'subscription_end_date',
            'ai_chats_used_today', 'has_active_subscription'
        ]
        read_only_fields = fields

class UserDetailSerializer(serializers.ModelSerializer):
    profile = UserProfileSerializer(read_only=True)
    active_subscription = serializers.SerializerMethodField()
    
    class Meta:
        model = User
        fields = ['id', 'email', 'first_name', 'last_name', 'profile', 'active_subscription', 'date_joined']
        read_only_fields = fields
    
    def get_active_subscription(self, obj):
        active_sub = Subscription.objects.filter(
            user=obj, 
            is_active=True, 
            end_date__gte=serializers.DateTimeField().to_representation(timezone.now())
        ).first()
        
        if active_sub:
            return SubscriptionSerializer(active_sub).data
        return None

class ReferralSerializer(serializers.Serializer):
    referral_code = serializers.CharField(
        max_length=16,
        min_length=16,
        required=True,
        help_text="Enter the 16-character referral code"
    )
    
    def validate_referral_code(self, value):
        if not UserProfile.objects.filter(referral_code=value).exists():
            raise serializers.ValidationError("Invalid referral code")
        
        # Check if this is the user's own referral code
        user = self.context['request'].user
        if user.profile.referral_code == value:
            raise serializers.ValidationError("You cannot use your own referral code")
            
        # Check if user already has a referrer
        if user.profile.referred_by is not None:
            raise serializers.ValidationError("You have already used a referral code")
            
        return value

class CreditHistorySerializer(serializers.ModelSerializer):
    class Meta:
        model = CreditTransaction
        fields = ['id', 'amount', 'transaction_type', 'description', 'balance_after', 'created_at']
        read_only_fields = fields

In [ ]:
from rest_framework import viewsets, permissions, status, mixins, generics
from rest_framework.response import Response
from rest_framework.decorators import action, api_view, permission_classes, throttle_classes
from rest_framework.throttling import UserRateThrottle, AnonRateThrottle
from rest_framework.exceptions import ValidationError, PermissionDenied
from django.utils import timezone
from django.db import transaction
from django.shortcuts import get_object_or_404
from rest_framework.pagination import PageNumberPagination  # Added import for PageNumberPagination

from django.conf import settings
from .models import (
    UserProfile, CreditTransaction, SubscriptionPlan, Subscription
)
from .serializers import (
    UserDetailSerializer, UserProfileSerializer, CreditTransactionSerializer,
    SubscriptionPlanSerializer, ReferralSerializer, CreditHistorySerializer
)
from .permissions import IsOwnerOrReadOnly
from .throttling import UserProfileRateThrottle
import razorpay
import logging

logger = logging.getLogger(__name__)

class StandardResultsSetPagination(PageNumberPagination):
    page_size = 10
    page_size_query_param = 'page_size'
    max_page_size = 100

class UserProfileView(generics.RetrieveUpdateAPIView):
    """
    Retrieve or update the authenticated user's profile
    """
    serializer_class = UserDetailSerializer
    permission_classes = [permissions.IsAuthenticated]
    throttle_classes = [UserProfileRateThrottle]
    
    def get_object(self):
        user = self.request.user
        
        # Update streak when profile is accessed
        user.profile.update_streak()
        
        return user
    
    def retrieve(self, request, *args, **kwargs):
        user = self.get_object()
        serializer = self.get_serializer(user)
        return Response(serializer.data)
    
    def update(self, request, *args, **kwargs):
        # Only allow partial updates to first_name and last_name
        user = self.get_object()
        
        if 'first_name' in request.data:
            user.first_name = request.data['first_name']
        
        if 'last_name' in request.data:
            user.last_name = request.data['last_name']
        
        user.save()
        serializer = self.get_serializer(user)
        return Response(serializer.data)

class SubscriptionPlanViewSet(mixins.ListModelMixin, viewsets.GenericViewSet):
    """
    List available subscription plans
    """
    queryset = SubscriptionPlan.objects.filter(is_active=True)
    serializer_class = SubscriptionPlanSerializer
    permission_classes = [permissions.IsAuthenticated]

class CreditHistoryView(generics.ListAPIView):
    """
    List credit transaction history for the authenticated user
    """
    serializer_class = CreditHistorySerializer
    permission_classes = [permissions.IsAuthenticated]
    pagination_class = StandardResultsSetPagination
    throttle_classes = [UserRateThrottle]
    
    def get_queryset(self):
        return CreditTransaction.objects.filter(user=self.request.user).order_by('-created_at')

class ReferralCodeView(generics.CreateAPIView):
    """
    Apply a referral code to receive bonus credits
    """
    serializer_class = ReferralSerializer
    permission_classes = [permissions.IsAuthenticated]
    throttle_classes = [UserRateThrottle]
    
    @transaction.atomic
    def create(self, request, *args, **kwargs):
        serializer = self.get_serializer(data=request.data)
        serializer.is_valid(raise_exception=True)
        
        referral_code = serializer.validated_data['referral_code']
        user = request.user
        
        # Get the referrer profile
        referrer_profile = get_object_or_404(UserProfile, referral_code=referral_code)
        
        # Set up the referral relationship
        user_profile = user.profile
        user_profile.referred_by = referrer_profile
        user_profile.save()
        
        # Add bonus credits to both users
        referral_bonus = 50
        
        # Credit the referred user
        user_profile.credits += referral_bonus
        user_profile.save()
        
        # Record transaction for referred user
        CreditTransaction.objects.create(
            user=user,
            amount=referral_bonus,
            transaction_type='REFERRAL',
            description=f'Bonus for using referral code',
            balance_after=user_profile.credits
        )
        
        # Credit the referrer
        referrer_profile.credits += referral_bonus
        referrer_profile.save()
        
        # Record transaction for referrer
        CreditTransaction.objects.create(
            user=referrer_profile.user,
            amount=referral_bonus,
            transaction_type='REFERRAL',
            description=f'Bonus for referral: {user.email}',
            balance_after=referrer_profile.credits
        )
        
        return Response(
            {"message": f"Referral applied successfully! You and the referrer both received {referral_bonus} credits."},
            status=status.HTTP_200_OK
        )

class AICreditsCheckView(generics.GenericAPIView):
    """
    Check if user can use AI chat, and consume a credit if they can
    """
    permission_classes = [permissions.IsAuthenticated]
    throttle_classes = [UserRateThrottle]
    
    @transaction.atomic
    def post(self, request):
        user = request.user
        profile = user.profile
        
        # Check if user can use AI chat
        if not profile.can_use_ai_chat():
            return Response(
                {"error": "You don't have enough credits or an active subscription to use AI chat."},
                status=status.HTTP_403_FORBIDDEN
            )
        
        # Determine if using subscription or credits
        today = timezone.now().date()
        using_subscription = False
        
        # Check if AI chat limit reset is needed
        if profile.ai_chats_last_reset != today:
            profile.ai_chats_used_today = 0
            profile.ai_chats_last_reset = today
            
        # If user has active subscription with available chats
        if (
            profile.has_active_subscription() and 
            profile.subscription and 
            profile.ai_chats_used_today < profile.subscription.ai_chat_limit
        ):
            using_subscription = True
            profile.ai_chats_used_today += 1
            profile.save()
            
            return Response({
                "success": True,
                "message": f"Using AI chat from your subscription. {profile.subscription.ai_chat_limit - profile.ai_chats_used_today} chats remaining today.",
                "chats_remaining": profile.subscription.ai_chat_limit - profile.ai_chats_used_today,
                "used_credits": False
            })
        
        # Otherwise, use credits
        if profile.credits > 0:
            profile.credits -= 1
            profile.save()
            
            # Record the transaction
            CreditTransaction.objects.create(
                user=user,
                amount=-1,
                transaction_type='AI_CHAT',
                description='Used 1 credit for AI chat',
                balance_after=profile.credits
            )
            
            return Response({
                "success": True,
                "message": f"Using 1 credit for AI chat. {profile.credits} credits remaining.",
                "credits_remaining": profile.credits,
                "used_credits": True
            })
        
        # This shouldn't happen due to the earlier check, but just in case
        return Response(
            {"error": "You don't have enough credits or an active subscription to use AI chat."},
            status=status.HTTP_403_FORBIDDEN
        )

# Initialize Razorpay client
razorpay_client = razorpay.Client(
    auth=(settings.RAZORPAY_KEY_ID, settings.RAZORPAY_KEY_SECRET)
) if hasattr(settings, 'RAZORPAY_KEY_ID') else None

class SubscriptionCreateView(generics.CreateAPIView):
    """
    Create a Razorpay order for subscription purchase
    """
    permission_classes = [permissions.IsAuthenticated]
    throttle_classes = [UserRateThrottle]
    
    def create(self, request, *args, **kwargs):
        if not razorpay_client:
            return Response(
                {"error": "Payment gateway not configured"},
                status=status.HTTP_501_NOT_IMPLEMENTED
            )
        
        # Get the plan ID from request data
        plan_id = request.data.get('plan_id')
        if not plan_id:
            return Response(
                {"error": "Plan ID is required"},
                status=status.HTTP_400_BAD_REQUEST
            )
        
        try:
            plan = SubscriptionPlan.objects.get(id=plan_id, is_active=True)
        except SubscriptionPlan.DoesNotExist:
            return Response(
                {"error": "Invalid subscription plan"},
                status=status.HTTP_400_BAD_REQUEST
            )
        
        # Create Razorpay order
        amount = int(plan.price * 100)  # Convert to paise (smallest Indian currency unit)
        currency = "INR"
        
        # Prepare order data
        order_data = {
            "amount": amount,
            "currency": currency,
            "receipt": f"subscription_{request.user.id}_{plan.id}_{timezone.now().timestamp()}",
            "notes": {
                "user_id": str(request.user.id),
                "plan_id": str(plan.id),
                "plan_name": plan.get_name_display()
            }
        }
        
        try:
            order = razorpay_client.order.create(data=order_data)
            
            # Return the order ID and amount to frontend
            return Response({
                "order_id": order["id"],
                "amount": amount / 100,  # Convert back to rupees for display
                "currency": currency,
                "plan": SubscriptionPlanSerializer(plan).data
            })
        except Exception as e:
            logger.error(f"Razorpay order creation failed: {str(e)}")
            return Response(
                {"error": "Failed to create payment order"},
                status=status.HTTP_500_INTERNAL_SERVER_ERROR
            )

class VerifySubscriptionView(generics.CreateAPIView):
    """
    Verify Razorpay payment and activate subscription
    """
    permission_classes = [permissions.IsAuthenticated]
    throttle_classes = [UserRateThrottle]
    
    @transaction.atomic
    def create(self, request, *args, **kwargs):
        if not razorpay_client:
            return Response(
                {"error": "Payment gateway not configured"},
                status=status.HTTP_501_NOT_IMPLEMENTED
            )
        
        # Get payment verification data
        payment_id = request.data.get('razorpay_payment_id')
        order_id = request.data.get('razorpay_order_id')
        signature = request.data.get('razorpay_signature')
        plan_id = request.data.get('plan_id')
        
        if not all([payment_id, order_id, signature, plan_id]):
            return Response(
                {"error": "Missing required payment verification data"},
                status=status.HTTP_400_BAD_REQUEST
            )
        
        # Verify payment signature
        params_dict = {
            'razorpay_payment_id': payment_id,
            'razorpay_order_id': order_id,
            'razorpay_signature': signature
        }
        
        try:
            razorpay_client.utility.verify_payment_signature(params_dict)
        except Exception as e:
            logger.error(f"Payment verification failed: {str(e)}")
            return Response(
                {"error": "Payment verification failed"},
                status=status.HTTP_400_BAD_REQUEST
            )
        
        # Get the subscription plan
        try:
            plan = SubscriptionPlan.objects.get(id=plan_id, is_active=True)
        except SubscriptionPlan.DoesNotExist:
            return Response(
                {"error": "Invalid subscription plan"},
                status=status.HTTP_400_BAD_REQUEST
            )
        
        user = request.user
        
        # Set subscription duration based on plan (1 month by default)
        duration_days = 30
        if plan.name in ['TEAM', 'ENTERPRISE']:
            duration_days = 365  # 1 year for business plans
        
        # Create new subscription
        subscription = Subscription.objects.create(
            user=user,
            plan=plan,
            start_date=timezone.now(),
            end_date=timezone.now() + timezone.timedelta(days=duration_days),
            is_active=True,
            razorpay_payment_id=payment_id,
            razorpay_order_id=order_id,
            razorpay_signature=signature
        )
        
        # Update user profile
        profile = user.profile
        profile.subscription = plan
        profile.subscription_start_date = subscription.start_date
        profile.subscription_end_date = subscription.end_date
        profile.save()
        
        return Response({
            "success": True,
            "message": f"Subscription to {plan.get_name_display()} activated successfully",
            "subscription": SubscriptionSerializer(subscription).data
        })

In [ ]:
from rest_framework import permissions

class IsOwnerOrReadOnly(permissions.BasePermission):
    """
    Custom permission to only allow owners of an object to edit it.
    """
    def has_object_permission(self, request, view, obj):
        # Read permissions are allowed to any request,
        # so we'll always allow GET, HEAD or OPTIONS requests.
        if request.method in permissions.SAFE_METHODS:
            return True
        
        # Write permissions are only allowed to the owner of the object
        return obj.user == request.user

In [ ]:
from rest_framework.throttling import UserRateThrottle

class UserProfileRateThrottle(UserRateThrottle):
    rate = '10/minute'
    scope = 'user_profile'

class CreditHistoryRateThrottle(UserRateThrottle):
    rate = '30/minute'
    scope = 'credit_history'

class ReferralRateThrottle(UserRateThrottle):
    rate = '5/day'
    scope = 'referral'

class SubscriptionRateThrottle(UserRateThrottle):
    rate = '10/hour'
    scope = 'subscription'

In [ ]:
from django.urls import path, include
from rest_framework.routers import DefaultRouter
from . import views

router = DefaultRouter()
router.register(r'subscription-plans', views.SubscriptionPlanViewSet)

urlpatterns = [
    # User profile endpoint
    path('profile/', views.UserProfileView.as_view(), name='user-profile'),
    
    # Credit management endpoints
    path('credits/history/', views.CreditHistoryView.as_view(), name='credit-history'),
    path('credits/check-ai/', views.AICreditsCheckView.as_view(), name='check-ai-credits'),
    
    # Referral system endpoint
    path('referral/apply/', views.ReferralCodeView.as_view(), name='apply-referral'),
    
    # Subscription endpoints
    path('subscription/create/', views.SubscriptionCreateView.as_view(), name='create-subscription'),
    path('subscription/verify/', views.VerifySubscriptionView.as_view(), name='verify-subscription'),
    
    # Include router URLs
    path('', include(router.urls)),
]

In [ ]:
import re
from django.conf import settings
from django.http import HttpResponseForbidden
import logging

logger = logging.getLogger(__name__)

class SecurityMiddleware:
    """
    Custom security middleware for additional protection.
    """
    def __init__(self, get_response):
        self.get_response = get_response
        # Compile regex for dangerous SQL patterns
        self.sql_patterns = re.compile(
            r'(\b(select|update|insert|delete|drop|alter|union|exec|execute)\b|--)',
            re.IGNORECASE
        )

    def __call__(self, request):
        # Check for SQL injection in URL parameters
        query_string = request.META.get('QUERY_STRING', '')
        if self.sql_patterns.search(query_string):
            logger.warning(f"Potential SQL injection attempt from {request.META.get('REMOTE_ADDR')}")
            return HttpResponseForbidden("Forbidden")
            
        # Add security headers
        response = self.get_response(request)
        response['X-Content-Type-Options'] = 'nosniff'
        response['X-Frame-Options'] = 'DENY'
        response['X-XSS-Protection'] = '1; mode=block'
        response['Referrer-Policy'] = 'strict-origin-when-cross-origin'
        response['Permissions-Policy'] = 'geolocation=(), camera=(), microphone=()'
        
        if not settings.DEBUG:
            response['Strict-Transport-Security'] = 'max-age=31536000; includeSubDomains'
            
        return response

class ContentSecurityPolicyMiddleware:
    """
    Middleware to add Content-Security-Policy headers
    """
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        response = self.get_response(request)
        
        # Skip for API endpoints (they don't need CSP headers)
        if '/api/' in request.path:
            return response
            
        # Add CSP headers for non-API requests
        csp_directives = [
            "default-src 'self'",
            "script-src 'self' https://cdnjs.cloudflare.com https://js.stripe.com",
            "style-src 'self' https://cdnjs.cloudflare.com 'unsafe-inline'",
            "img-src 'self' data: https:",
            "font-src 'self' https://cdnjs.cloudflare.com",
            "connect-src 'self' https://api.razorpay.com",
            "frame-src https://checkout.razorpay.com",
            "object-src 'none'",
            "base-uri 'self'",
            "form-action 'self'",
        ]
        
        response['Content-Security-Policy'] = "; ".join(csp_directives)
        return response

In [ ]:
from django.contrib.auth import get_user_model
from django.utils.functional import SimpleLazyObject
from django.utils.deprecation import MiddlewareMixin
from rest_framework_simplejwt.authentication import JWTAuthentication
from rest_framework_simplejwt.exceptions import InvalidToken, TokenError
from django.contrib.auth.models import AnonymousUser
import logging

logger = logging.getLogger(__name__)

User = get_user_model()

def get_user_jwt(request):
    """
    Extract and authenticate the user from JWT token in the request
    """
    # If we already have an authenticated user from session or basic auth, use it
    if hasattr(request, "_jwt_user"):
        return request._jwt_user
        
    # Try to authenticate using JWT
    jwt_auth = JWTAuthentication()
    request._jwt_user = AnonymousUser()
    
    try:
        if 'HTTP_AUTHORIZATION' in request.META:
            user_auth_tuple = jwt_auth.authenticate(request)
            if user_auth_tuple is not None:
                request._jwt_user = user_auth_tuple[0]
    except InvalidToken:
        logger.warning(f"Invalid JWT token received from {request.META.get('REMOTE_ADDR')}")
    except TokenError:
        logger.warning(f"JWT token error from {request.META.get('REMOTE_ADDR')}")
    except Exception as e:
        logger.error(f"JWT authentication error: {str(e)}")
        
    return request._jwt_user

class JWTAuthenticationMiddleware(MiddlewareMixin):
    def process_request(self, request):
        """
        Process each request to add user from JWT token to request object
        """
        # Don't override existing user from session
        if not hasattr(request, "user") or request.user.is_anonymous:
            request.user = SimpleLazyObject(lambda: get_user_jwt(request))

In [ ]:
# Add these to your Django settings.py file

# Security Settings
SECURE_BROWSER_XSS_FILTER = True
SECURE_CONTENT_TYPE_NOSNIFF = True
SESSION_COOKIE_SECURE = True
CSRF_COOKIE_SECURE = True
CSRF_COOKIE_HTTPONLY = True
X_FRAME_OPTIONS = 'DENY'

# Add Security & JWT Middleware
MIDDLEWARE += [
    'your_app.middleware.SecurityMiddleware',
    'your_app.middleware.ContentSecurityPolicyMiddleware',
    'your_app.jwt_middleware.JWTAuthenticationMiddleware',
]

# DRF Settings
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': (
        'rest_framework_simplejwt.authentication.JWTAuthentication',
        'rest_framework.authentication.SessionAuthentication',
    ),
    'DEFAULT_PERMISSION_CLASSES': (
        'rest_framework.permissions.IsAuthenticated',
    ),
    'DEFAULT_THROTTLE_CLASSES': [
        'rest_framework.throttling.AnonRateThrottle',
        'rest_framework.throttling.UserRateThrottle',
    ],
    'DEFAULT_THROTTLE_RATES': {
        'anon': '60/hour',
        'user': '1000/day',
        'user_profile': '10/minute',
        'credit_history': '30/minute',
        'referral': '5/day',
        'subscription': '10/hour',
    },
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.PageNumberPagination',
    'PAGE_SIZE': 10,
    'DEFAULT_SCHEMA_CLASS': 'rest_framework.schemas.coreapi.AutoSchema',
    'EXCEPTION_HANDLER': 'your_app.utils.custom_exception_handler',
}

# JWT Settings
from datetime import timedelta
SIMPLE_JWT = {
    'ACCESS_TOKEN_LIFETIME': timedelta(minutes=60),
    'REFRESH_TOKEN_LIFETIME': timedelta(days=14),
    'ROTATE_REFRESH_TOKENS': True,
    'BLACKLIST_AFTER_ROTATION': True,
    'ALGORITHM': 'HS256',
    'SIGNING_KEY': SECRET_KEY,
    'VERIFYING_KEY': None,
    'AUTH_HEADER_TYPES': ('Bearer',),
    'USER_ID_FIELD': 'id',
    'USER_ID_CLAIM': 'user_id',
    'AUTH_TOKEN_CLASSES': ('rest_framework_simplejwt.tokens.AccessToken',),
    'TOKEN_TYPE_CLAIM': 'token_type',
    'JTI_CLAIM': 'jti',
    'SLIDING_TOKEN_REFRESH_EXP_CLAIM': 'refresh_exp',
    'SLIDING_TOKEN_LIFETIME': timedelta(minutes=60),
    'SLIDING_TOKEN_REFRESH_LIFETIME': timedelta(days=14),
}

# Database Configuration for Optimized Queries
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'your_db_name',
        'USER': 'your_db_user',
        'PASSWORD': 'your_db_password',
        'HOST': 'localhost',
        'PORT': '5432',
        'CONN_MAX_AGE': 600,  # Persistent connections
        'OPTIONS': {
            'sslmode': 'require',  # For production
        },
        'ATOMIC_REQUESTS': True,  # All views become transactions
    }
}

# Razorpay Payment Integration
RAZORPAY_KEY_ID = 'your_razorpay_key_id'  # Replace with actual key
RAZORPAY_KEY_SECRET = 'your_razorpay_key_secret'  # Replace with actual secret

# Redis Cache for Better Performance
CACHES = {
    'default': {
        'BACKEND': 'django_redis.cache.RedisCache',
        'LOCATION': 'redis://127.0.0.1:6379/1',
        'OPTIONS': {
            'CLIENT_CLASS': 'django_redis.client.DefaultClient',
            'PARSER_CLASS': 'redis.connection.HiredisParser',
            'CONNECTION_POOL_KWARGS': {'max_connections': 100},
            'SOCKET_CONNECT_TIMEOUT': 5,
            'SOCKET_TIMEOUT': 5,
        }
    }
}

# Session Cache
SESSION_ENGINE = 'django.contrib.sessions.backends.cache'
SESSION_CACHE_ALIAS = 'default'

# Logging Configuration
LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'formatters': {
        'verbose': {
            'format': '{levelname} {asctime} {module} {process:d} {thread:d} {message}',
            'style': '{',
        },
    },
    'handlers': {
        'file': {
            'level': 'INFO',
            'class': 'logging.FileHandler',
            'filename': '/path/to/your/logs/django.log',
            'formatter': 'verbose',
        },
        'console': {
            'level': 'INFO',
            'class': 'logging.StreamHandler',
            'formatter': 'verbose',
        },
    },
    'loggers': {
        'django': {
            'handlers': ['file', 'console'],
            'level': 'INFO',
            'propagate': True,
        },
        'your_app': {
            'handlers': ['file', 'console'],
            'level': 'INFO',
            'propagate': True,
        },
    },
}

In [ ]:
from django.db import models
from django.utils import timezone
import hashlib
import uuid
import logging

logger = logging.getLogger(__name__)

class AuditableMixin(models.Model):
    """
    A mixin for adding audit fields to models
    """
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    created_by = models.ForeignKey(
        'auth.User', 
        related_name='%(class)s_created',
        null=True, 
        blank=True,
        on_delete=models.SET_NULL
    )
    updated_by = models.ForeignKey(
        'auth.User', 
        related_name='%(class)s_updated',
        null=True, 
        blank=True,
        on_delete=models.SET_NULL
    )
    
    class Meta:
        abstract = True

class IPTrackingMixin(models.Model):
    """
    A mixin for tracking IP addresses
    """
    ip_address = models.GenericIPAddressField(null=True, blank=True)
    user_agent = models.TextField(null=True, blank=True)
    
    class Meta:
        abstract = True

class SecurityHashMixin(models.Model):
    """
    A mixin for adding security hashes to models to detect tampering
    """
    security_hash = models.CharField(max_length=64, null=True, blank=True)
    hash_updated_at = models.DateTimeField(null=True, blank=True)
    
    def generate_security_hash(self):
        """
        Generate a hash based on critical fields to detect tampering
        """
        # This should be customized per model, this is a basic example
        # Combine important fields that shouldn't be changed
        hash_input = f"{self.id}:{self.created_at}"
        
        # Add model-specific sensitive fields
        if hasattr(self, 'user_id'):
            hash_input += f":{self.user_id}"
        
        if hasattr(self, 'amount'):
            hash_input += f":{self.amount}"
            
        # Create hash
        hash_obj = hashlib.sha256(hash_input.encode())
        self.security_hash = hash_obj.hexdigest()
        self.hash_updated_at = timezone.now()
    
    def verify_security_hash(self):
        """
        Verify that the security hash is valid to detect tampering
        """
        if not self.security_hash:
            return False
            
        current_hash = self.security_hash
        self.generate_security_hash()
        is_valid = current_hash == self.security_hash
        
        # Restore original hash
        self.security_hash = current_hash
        
        if not is_valid:
            logger.warning(f"Security hash mismatch detected for {self.__class__.__name__} id={self.id}")
            
        return is_valid
    
    def save(self, *args, **kwargs):
        """
        Override save to update the security hash
        """
        result = super().save(*args, **kwargs)
        self.generate_security_hash()
        return result
    
    class Meta:
        abstract = True

class AntiTamperingMixin(models.Model):
    """
    A mixin for adding tamper-evident features to models
    """
    uuid = models.UUIDField(default=uuid.uuid4, editable=False)
    modified_count = models.PositiveIntegerField(default=0, editable=False)
    last_modified = models.DateTimeField(auto_now=True)
    
    def save(self, *args, **kwargs):
        if self.pk is not None:
            self.modified_count += 1
        super().save(*args, **kwargs)
    
    class Meta:
        abstract = True

In [ ]:
from rest_framework.views import exception_handler
from rest_framework.response import Response
from rest_framework import status
from django.http import JsonResponse
from django.db import transaction
from django.utils import timezone
from .models import UserProfile, CreditTransaction
import logging

logger = logging.getLogger(__name__)

def custom_exception_handler(exc, context):
    """
    Custom exception handler for DRF to standardize error responses
    """
    # Call REST framework's default exception handler first
    response = exception_handler(exc, context)
    
    # Log detailed exception info
    logger.error(f"Exception: {str(exc)}, Context: {context['view'].__class__.__name__}")
    
    # If the response is None, it's an unhandled exception
    if response is None:
        if isinstance(exc, Exception):
            logger.exception("Unhandled exception", exc_info=exc)
            return Response(
                {"error": "An unexpected error occurred."},
                status=status.HTTP_500_INTERNAL_SERVER_ERROR
            )
        return None
    
    # Standardize the response format
    if isinstance(response.data, dict):
        if 'detail' in response.data:
            response.data = {"error": response.data['detail']}
        elif any(isinstance(v, list) for v in response.data.values()):
            # For validation errors that return lists
            field_errors = {}
            for field, errors in response.data.items():
                if isinstance(errors, list):
                    field_errors[field] = errors[0]
                else:
                    field_errors[field] = errors
            response.data = {"field_errors": field_errors}
    
    return response

def get_client_ip(request):
    """
    Get the client IP address from the request
    """
    x_forwarded_for = request.META.get('HTTP_X_FORWARDED_FOR')
    if x_forwarded_for:
        ip = x_forwarded_for.split(',')[0]
    else:
        ip = request.META.get('REMOTE_ADDR')
    return ip

@transaction.atomic
def update_streak_and_award_bonus(user):
    """
    Update user streak and award bonus points if applicable
    """
    profile = user.profile
    today = timezone.now().date()
    
    # Initialize if first visit
    if not profile.last_active_date:
        profile.last_active_date = today
        profile.current_streak = 1
        profile.highest_streak = 1
        profile.save()
        return
    
    # Calculate days since last activity
    days_since_last_active = (today - profile.last_active_date).days
    
    # Update streak only if not already updated today
    if days_since_last_active == 0:
        # Already updated today
        return
    elif days_since_last_active == 1:
        # Consecutive day, increase streak
        profile.current_streak += 1
        
        # Update highest streak if needed
        if profile.current_streak > profile.highest_streak:
            profile.highest_streak = profile.current_streak
            
        # Award streak bonus at specific milestones
        bonus = 0
        description = ""
        
        if profile.current_streak == 7:  # 7-day streak
            bonus = 20
            description = "7-day streak bonus"
        elif profile.current_streak == 30:  # 30-day streak
            bonus = 100
            description = "30-day streak bonus"
        elif profile.current_streak == 100:  # 100-day streak
            bonus = 500
            description = "100-day streak bonus"
        elif profile.current_streak % 30 == 0:  # Every 30 days
            bonus = 50
            description = f"{profile.current_streak}-day streak bonus"
        
        # Award bonus if applicable
        if bonus > 0:
            profile.credits += bonus
            
            # Record the transaction
            CreditTransaction.objects.create(
                user=user,
                amount=bonus,
                transaction_type='STREAK',
                description=description,
                balance_after=profile.credits
            )
    else:
        # Streak broken
        profile.current_streak = 1
    
    # Update last active date
    profile.last_active_date = today
    profile.save()

def award_content_credits(user, content_type, content_id):
    """
    Award credits for content creation or contribution
    
    Args:
        user: User who created/contributed to content
        content_type: String describing the content (e.g., 'tutorial', 'article')
        content_id: ID of the content created/contributed to
    """
    # Define credit amounts for different content types
    credit_amounts = {
        'tutorial': 50,
        'article': 25,
        'code_snippet': 10,
        'comment': 1,
        'documentation': 20,
        'blog_post': 30,
        'question': 5,
        'answer': 15,
        'review': 10,
    }
    
    # Get credit amount for the content type (default to 5)
    amount = credit_amounts.get(content_type.lower(), 5)
    
    # Get user profile
    profile = user.profile
    
    with transaction.atomic():
        # Add credits
        profile.credits += amount
        profile.save()
        
        # Record the transaction
        CreditTransaction.objects.create(
            user=user,
            amount=amount,
            transaction_type='USAGE',
            description=f"Earned {amount} credits for {content_type} contribution (ID: {content_id})",
            balance_after=profile.credits
        )
        
    return amount

def check_access_permission(user, content_type):
    """
    Check if user has permission to access specific content type
    based on their subscription plan or credit balance
    
    Args:
        user: User trying to access content
        content_type: String representing content type ('tutorial', 'library', 'news', 'premium')
    
    Returns:
        tuple: (has_access, reason)
    """
    profile = user.profile
    
    # Check if user has active subscription
    if profile.has_active_subscription() and profile.subscription:
        plan = profile.subscription
        
        # Check feature access based on subscription plan
        if content_type == 'tutorial' and plan.access_tutorials:
            return True, "Access granted through subscription"
        elif content_type == 'library' and plan.access_library:
            return True, "Access granted through subscription"
        elif content_type == 'news' and plan.access_news:
            return True, "Access granted through subscription"
        elif content_type == 'premium' and plan.access_premium:
            return True, "Access granted through subscription"
    
    # If no subscription access, check credits
    if profile.credits > 0:
        # Define credit costs for content types
        credit_costs = {
            'tutorial': 1,
            'library': 1,
            'news': 2,
            'premium': 5,
        }
        
        cost = credit_costs.get(content_type, 0)
        
        if profile.credits >= cost:
            return True, f"Access granted using {cost} credits"
    
    # No access
    return False, "Insufficient credits or subscription level"

@transaction.atomic
def use_content_credits(user, content_type):
    """
    Use credits for accessing specific content
    
    Args:
        user: User accessing content
        content_type: Type of content being accessed
    
    Returns:
        bool: Whether the credits were successfully used
    """
    profile = user.profile
    
    # Define credit costs for content types
    credit_costs = {
        'tutorial': 1,
        'library': 1,
        'news': 2,
        'premium': 5,
    }
    
    cost = credit_costs.get(content_type, 0)
    
    # Check if user has sufficient credits
    if profile.credits >= cost:
        # Deduct credits
        profile.credits -= cost
        profile.save()
        
        # Record the transaction
        CreditTransaction.objects.create(
            user=user,
            amount=-cost,
            transaction_type='USAGE',
            description=f"Used {cost} credits to access {content_type} content",
            balance_after=profile.credits
        )
        
        return True
    
    return False

In [ ]:
"""
PostgreSQL Indexing Strategy for Credits and Subscription System

This file contains recommendations for PostgreSQL indexes and query optimization
for the credit system and subscription platform. These are not actual models,
but SQL statements to run in your PostgreSQL database to optimize performance.

Run these after your initial migrations to improve query performance, especially
for high write throughput scenarios.
"""

# Index recommendations for UserProfile model
"""
-- Add indexes for frequent query patterns on UserProfile
CREATE INDEX idx_userprofile_user_id ON your_app_userprofile(user_id);
CREATE INDEX idx_userprofile_subscription_id ON your_app_userprofile(subscription_id);
CREATE INDEX idx_userprofile_referral_code ON your_app_userprofile(referral_code);
CREATE INDEX idx_userprofile_subscription_end_date ON your_app_userprofile(subscription_end_date);
CREATE INDEX idx_userprofile_credits ON your_app_userprofile(credits);

-- For efficient streak calculations
CREATE INDEX idx_userprofile_last_active_date ON your_app_userprofile(last_active_date);
CREATE INDEX idx_userprofile_current_streak ON your_app_userprofile(current_streak);
"""

# Index recommendations for CreditTransaction model
"""
-- Composite index for common query patterns (user + time range)
CREATE INDEX idx_credittransaction_user_created_at ON your_app_credittransaction(user_id, created_at DESC);

-- Index for transaction type filtering
CREATE INDEX idx_credittransaction_transaction_type ON your_app_credittransaction(transaction_type);

-- For time-based analytics
CREATE INDEX idx_credittransaction_created_at ON your_app_credittransaction(created_at);

-- For balance history queries
CREATE INDEX idx_credittransaction_balance_after ON your_app_credittransaction(balance_after);
"""

# Index recommendations for Subscription model
"""
-- For subscription management
CREATE INDEX idx_subscription_user_is_active ON your_app_subscription(user_id, is_active);
CREATE INDEX idx_subscription_end_date ON your_app_subscription(end_date);
CREATE INDEX idx_subscription_plan_id ON your_app_subscription(plan_id);

-- For payment reconciliation
CREATE INDEX idx_subscription_razorpay_payment_id ON your_app_subscription(razorpay_payment_id);
CREATE INDEX idx_subscription_razorpay_order_id ON your_app_subscription(razorpay_order_id);
"""

# Partial indexes for performance optimization
"""
-- Partial index for active subscriptions only (improves query performance)
CREATE INDEX idx_subscription_active_end_date 
ON your_app_subscription(end_date) 
WHERE is_active = true;

-- Partial index for users with high credits (for special rewards/targeting)
CREATE INDEX idx_userprofile_high_credits 
ON your_app_userprofile(credits) 
WHERE credits > 1000;

-- Partial index for recent transactions (improves dashboard performance)
CREATE INDEX idx_credittransaction_recent 
ON your_app_credittransaction(user_id, created_at) 
WHERE created_at > NOW() - INTERVAL '30 days';
"""

# Additional performance optimizations
"""
-- Enable PostgreSQL query planner statistics for better execution plans
ANALYZE your_app_userprofile;
ANALYZE your_app_credittransaction;
ANALYZE your_app_subscription;
ANALYZE your_app_subscriptionplan;

-- Increase VACUUM frequency for high-write tables
ALTER TABLE your_app_credittransaction SET (autovacuum_vacuum_scale_factor = 0.05);
ALTER TABLE your_app_credittransaction SET (autovacuum_analyze_scale_factor = 0.025);
"""

# Table partitioning for high-volume tables (run before creating the table)
"""
-- For very large credit transaction history, consider partitioning by time
CREATE TABLE your_app_credittransaction (
    id BIGSERIAL PRIMARY KEY,
    user_id INTEGER NOT NULL REFERENCES auth_user(id),
    amount INTEGER NOT NULL,
    transaction_type VARCHAR(15) NOT NULL,
    description TEXT NOT NULL,
    balance_after INTEGER NOT NULL,
    created_at TIMESTAMP WITH TIME ZONE NOT NULL
) PARTITION BY RANGE (created_at);

-- Create monthly partitions
CREATE TABLE your_app_credittransaction_y2024m01 PARTITION OF your_app_credittransaction